In [1]:
import numpy as np 
import pandas as pd
import sqlite3

In [33]:
conn = sqlite3.connect("../database.db")

stock = pd.read_sql("SELECT * FROM Ticker_1D WHERE Ticker = 'AAPL'", con=conn)
benchmark = pd.read_sql("SELECT * FROM Ticker_1D WHERE Ticker = '^GSPC'", con=conn)

stock["Datetime"] = pd.to_datetime(stock["Datetime"])
benchmark["Datetime"] = pd.to_datetime(benchmark["Datetime"])

len(stock), len(benchmark)

(2514, 2514)

In [41]:
pd.Timedelta(days=365.25 * 5)

Timedelta('1826 days 06:00:00')

In [36]:
look_back = pd.to_datetime("2021-02-13")
starting_index = stock[stock["Datetime"] >= look_back].index[0]

In [58]:
beta_df = pd.DataFrame(columns=["datetime", "beta"])

for i in range(len(stock) - starting_index):
    end_date = stock.iloc[i + starting_index, 1]
    start_date = end_date - pd.Timedelta(days=365.25 * 5)

    stock_mask = (stock["Datetime"] >= start_date) & (stock["Datetime"] <= end_date)
    benchmark_mask = (benchmark["Datetime"] >= start_date) & (benchmark["Datetime"] <= end_date)

    inner_stock = stock.iloc[stock_mask]
    inner_benchmark = benchmark.iloc[benchmark_mask]

    inner_stock_ret = inner_stock["Close"].pct_change().dropna()
    inner_benchmark_ret = inner_benchmark["Close"].pct_change().dropna()

    inner_returns_df = pd.DataFrame({
        "stock": inner_stock_ret,
        "benchmark": inner_benchmark_ret
    }).dropna()

    inner_cov = inner_returns_df.cov().iloc[0,1]
    inner_var = inner_returns_df["benchmark"].var()

    cur_beta = inner_cov / inner_var

    # print(cur_beta)

    out_df = pd.DataFrame({
        "datetime": [end_date],
        "beta": [cur_beta]
    })

    beta_df = pd.concat([beta_df, out_df])


In [59]:
beta_df

,datetime,beta
0,2021-02-16 00:00:00,1.174954
0,2021-02-17 00:00:00,1.174631
0,2021-02-18 00:00:00,1.174739
0,2021-02-19 00:00:00,1.175397
0,2021-02-22 00:00:00,1.175757
...,...,...
0,2026-02-06 00:00:00,1.242861
0,2026-02-09 00:00:00,1.242283
0,2026-02-10 00:00:00,1.242242
0,2026-02-11 00:00:00,1.242249
